## Analizing life expectancy and GDP

The goal of this project is to get a better understanding of life expectancy and if and how it is correlated with GDP.

Data provided by: [Codecademy](https://www.codecademy.com) 

*Data Scientist: Machine Learning course, Data Visualization Portfolio Project module.*

`all_data.csv`